In [1]:
import thermo
from thermo.chemical import Chemical

In [2]:
import math
import fluids
from fluids.units import *


Given:  A 12" Schedule 40 steel pipe 60' long, containing a standard gate valve 10' from the entrance, discharges 60 degF water to atmosphere from a reservoir.  The entrance projects inward into the reservoir and its center line is 12' below the water level in the reservoir.

Find:  The diameter of thni-plate orifice that must be centrally installed in the pipe to restrict the veocity of flow to 10' per second when the gate valve is wide open.

In [3]:
T = (60*u.degF).to(u.degK)

Assume atmospheric pressures

In [4]:
P = (1*u.atm).to(u.Pa)

Get the water properties from PubChem

In [5]:
water = Chemical('water', P=P.magnitude, T=T.magnitude)

# Solve using the fluids-built in functions exclusively

Only the following types of clean, new pipe have data available:
‘Plastic coated’
‘Carbon steel, honed bare’
‘Cr13, electropolished bare’
‘Cement lining’
‘Carbon steel, bare’
‘Fiberglass lining’
‘Cr13, bare’

In [6]:
mu = (water.mu*u.Pa*u.sec).to(u.cP)
rho = (water.rho*u.kilogram/u.meter**3).to(u.lb/u.feet**3)
d = (nearest_pipe(NPS=12*u.inch, schedule='40')[1]).to(u.inch)
v = 10*u.feet/u.sec

ft = 0.013 #Crane 410, A-27

mu, rho, d, v, ft

(<Quantity(1.134622271808897, 'centipoise')>,
 <Quantity(62.46920305865831, 'pound / foot ** 3')>,
 <Quantity(11.936220472440946, 'inch')>,
 <Quantity(10.0, 'foot / second')>,
 0.013)

# Re

In [7]:
Re = (124.0*d*v*rho/mu).magnitude

Re

814897.8622451683

In [8]:
D = d.to(u.meter)
e = roughness_Farshad('Carbon steel, bare', D=d)

f = friction_factor(Re=Re, eD=e/D)
f  #friciton factor for the pipe flow

<Quantity(0.013878069561963996, 'dimensionless')>

# Evaluate the know Ks

In [9]:
#pipe
L=60*u.feet
D = 12*u.inch

Kpipe = f*L/D.to(u.feet)

Kpipe

<Quantity(0.8326841737178398, 'dimensionless')>

In [10]:
#gate valve
Kgate = 8*ft

Kgate


0.104

In [11]:
#K entrance
Kentrance = 0.78 #Crane 410, A-30

#K exit
Kexit = 1 #Crane 410, A-30

# Total Required K

In [12]:
g = 32.2*u.feet/u.sec**2
hL = 12*u.feet

Kreq = 2*g*hL/v**2

Kreq

<Quantity(7.728000000000001, 'dimensionless')>

# K for orifice required

In [13]:
Korifice = Kreq - (Kpipe + Kgate + Kexit + Kentrance)

Korifice

<Quantity(5.011315826282161, 'dimensionless')>

In [14]:
#Assume
beta = 0.7
C = 0.7 #Crane 410, A-21

Cd = C*math.sqrt(1-beta**4)
Korifice_cal = (math.sqrt(1-(beta**4)*(1-Cd**2))/(Cd*beta**2)-1)**2

Cd, Korifice_cal

(0.6102057030215302, 4.335491251870889)

Korifice_cal too small

In [15]:
#pick a different beta
beta = 0.65
C = 0.67 #Crane 410, A-21

Cd = C*math.sqrt(1-beta**4)
Korifice_cal = (math.sqrt(1-(beta**4)*(1-Cd**2))/(Cd*beta**2)-1)**2

Cd, Korifice_cal

(0.6072631590793237, 7.136607740166733)

Korifice_cal is too large

In [16]:
#pick a different beta
beta = 0.68
C = 0.70 #Crane 410, A-21

Cd = C*math.sqrt(1-beta**4)
Korifice_cal = (math.sqrt(1-(beta**4)*(1-Cd**2))/(Cd*beta**2)-1)**2

Cd, Korifice_cal

(0.6206700070085551, 5.050270285298161)

Use beta = 0.68

# Orifice Size

In [17]:
Orifice_size = beta*d

Orifice_size

<Quantity(8.116629921259843, 'inch')>

# Alternative way to solve this problem without using Crane 410 Appendix

#Calculates the coefficient of discharge of the orifice 
#based on the geometry of the plate, measured pressures of the orifice, 
#mass flow rate through the orifice, and the density and viscosity of the fluid

D : float

    Upstream internal pipe diameter, [m]

Do : float

    Diameter of orifice at flow conditions, [m]

rho : float

    Density of fluid at P1, [kg/m^3]

mu : float

    Viscosity of fluid at P1, [Pa*s]

m : float

    Mass flow rate of fluid through the orifice, [kg/s]

taps : str

    The orientation of the taps; one of ‘corner’, ‘flange’, ‘D’, or ‘D/2’, [-]


In [18]:
from flow_meter import *  #something is not right with the pip installation of fluids
                        #I had to locally say the soure data and
                        #modify the imports to local

#assume a beta
beta = 0.8

D = (nearest_pipe(NPS=12*u.inch, schedule='40')[1])
Do = beta*D 
mu = (water.mu*u.Pa*u.sec)
rho = (water.rho*u.kilogram/u.meter**3)

v = (10*u.feet/u.sec).to(u.meter/u.sec)
A = math.pi*(D/2)**2
m = rho*v*A #mass flow rate

D, Do, rho, mu, m

(<Quantity(0.30318, 'meter')>,
 <Quantity(0.242544, 'meter')>,
 <Quantity(1000.660641195597, 'kilogram / meter ** 3')>,
 <Quantity(0.001134622271808897, 'pascal * second')>,
 <Quantity(220.18755011519298, 'kilogram / second')>)

In [19]:
Cd = C_Reader_Harris_Gallagher(D=d.magnitude, Do=Do.magnitude, rho=rho.magnitude, mu=mu.magnitude, m=m.magnitude, taps='flange')

Korifice = discharge_coefficient_to_K(D=D.magnitude, Do=Do.magnitude, C=Cd)

Cd, Korifice, Do.to(u.inch)

(0.5966246688067108, 1.5551913739827734, <Quantity(9.548976377952757, 'inch')>)

After choosing a few betas, we finally get a beta of 0.69 to give the required Korifice.  Do is 8.2 inches.